In [3]:
import pandas

In [1]:

import os
os.chdir("../")

In [2]:
os.getcwd()

'c:\\Users\\somit\\Downloads\\project_ineuron\\deep_cnn_model'

In [1]:
os.chdir("c:\\Users\\somit\\Downloads\\project_ineuron\\deep_cnn_model\\src")

In [2]:
pip install ipykernal


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ipykernal (from versions: none)
ERROR: No matching distribution found for ipykernal


In [2]:
import deepClassifier

In [3]:
from deepClassifier.entity.config_entity import DataIngestionConfig

In [4]:
from deepClassifier.utils import read_yaml , create_directories

In [8]:
from deepClassifier.constant import *

In [12]:
from deepClassifier.constant import *
from deepClassifier.utils import read_yaml , create_directories
from deepClassifier.entity.config_entity import DataIngestionConfig


In [5]:
from collections import namedtuple

In [23]:
DataIngestionConfig =namedtuple("DataIngestionConfig" , ["root_dir" , "source_url" , "local_data_file" , "unzip_dir"])

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [28]:
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)


In [29]:
try:

    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingetion =DataIngestion(config=data_ingestion_config)
    data_ingetion.download_file()
    data_ingetion.unzip_and_clean()
except Exception as e:
    raise  e   
